link:

http://insider.espn.com/nba/hollinger/statistics

In [1]:
import pandas as pd
import csv

In [19]:
from selenium.webdriver import Firefox
from bs4 import BeautifulSoup

url = 'https://www.teamrankings.com/nba/player-stat/nba-efficiency?season_id=216'
driver = Firefox()
driver.get(url)

In [20]:
soup = BeautifulSoup(driver.page_source)

In [6]:
headers = soup.find(_class='colhead')

In [24]:
headers = soup.find('thead').find('tr').find_all('th')
headers = [i.text for i in headers]

In [25]:
headers

['Rank', 'Player', 'Team', 'Pos', 'Value']

In [71]:
with open('top_per_0607.csv', 'w') as file:
    csv_writer = csv.writer(file, delimiter=',')
    csv_writer.writerow([header for header in headers])
#     for i in range(9):
        
    soup = BeautifulSoup(driver.page_source) 
    players = soup.find('table').find_all('tr')[1:]
    for player in players:
        csv_writer.writerow( 
            [item.text for item in player.find_all('td')]
        )
    
# driver.find_element_by_xpah(r'/html/body/main/div[2]/div/div[2]/div/div/nba-stat-table/div[3]/div/div/a[2]').click()

In [72]:
data = pd.read_csv('top_per_1617.csv')
data.tail()

,Rank,Player,Team,Pos,Value
95,96,Kentavious Caldwell-Pope,Detroit Pistons,SG,12.03
96,97,Courtney Lee,New York Knicks,SG,11.86
97,98,Rondae Hollis-Jefferson,Brooklyn Nets,SF,11.81
98,99,Alex Len,Phoenix Suns,C,11.81
99,100,Jordan Clarkson,Los Angeles Lakers,PG,11.80


Now, then merge the results (left joint) with nba phytho

In [88]:
import glob
files = glob.glob("top_per*csv")

drop_cols = ['Team', 'Pos']
phythro = pd.read_csv('phythro_nba.csv')

In [91]:
for file in files:
    data = pd.read_csv(file)
    data = data.drop(drop_cols, axis=1)
    data.rename(columns={'Value':'Efficiency'}, inplace=True)
    data = data.merge(phythro, how='left', on='Player')
    data.to_csv(file)

Done. Now we have enought data to do analysis.

In [1]:
import pandas as pd
import csv

In [178]:
import re

r = re.compile(r"([\w\s]+), (\w+)")

def get_team(el):
    m = r.match(el)
    if m == None:
        return float('NaN')
    else:
        return(m.group(2))
def get_player(el):
    m = r.match(el)
    if m == None:
        return float('NaN')
    else:
        return(m.group(1))
    

In [148]:
page_num = re.compile(r'(\d+) (\w+) (\d+)')

In [170]:
from selenium.webdriver import Firefox
from bs4 import BeautifulSoup

url = 'http://insider.espn.com/nba/hollinger/statistics'

driver = Firefox()
driver.get(url)

In [151]:
soup = BeautifulSoup(driver.page_source)

In [152]:
headers = soup.find('table').find(class_='colhead').find_all('td')
headers = [item.text for item in headers]
headers

['RK',
 'PLAYER',
 'GP',
 'MPG',
 'TS%',
 'AST',
 'TO',
 'USG',
 'ORR',
 'DRR',
 'REBR',
 'PER',
 'VA',
 'EWA']

In [189]:

page = 1


for year in range(2000, 2021):
    url = 'http://insider.espn.com/nba/hollinger/statistics/_/page/1/year/' + str(year)
    page_state = soup.find(class_='page-numbers').text
    max_page = int(page_num.match(page_state).group(3))

    with open('full_per_' + str(year) + '.csv', 'w') as file:

        csv_writer = csv.writer(file, delimiter=',')
        csv_writer.writerow([header for header in headers])


        for page in range(1, max_page+1):
            url = 'http://insider.espn.com/nba/hollinger/statistics/_/page/' + str(page) + '/year/' + str(year)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source) 
            players = soup.find('table').find_all('tr')[1:]
            players = soup.find('table').find_all('tr')

            for player in players[2:]:
                cells = player.find_all('td')
                cells = [item.text for item in cells]
                if cells[0] == 'RK':
                    continue
                else:
                    csv_writer.writerow(cells)


In [190]:
d = pd.read_csv('full_per_2000.csv')

In [183]:
d['TEAM'] = d['PLAYER'].apply(get_team)
d['PLAYER'] = d['PLAYER'].apply(get_player)

In [184]:
d

,RK,PLAYER,GP,MPG,TS%,AST,TO,USG,ORR,DRR,REBR,PER,VA,EWA,TEAM
0,1.0,Giannis Antetokounmpo,57,30.9,0.608,17.1,10.8,37.1,7.8,34.5,22.0,31.71,531.8,17.7,MIL
1,2.0,James Harden,61,36.7,0.616,18.6,11.3,36.7,3.0,15.3,9.1,28.39,598.5,20.0,HOU
2,3.0,Anthony Davis,55,34.3,0.614,11.5,9.0,29.2,7.3,22.1,14.9,28.25,472.1,15.7,LAL
3,4.0,Luka Doncic,54,33.3,0.584,23.2,11.2,38.5,4.1,24.7,14.6,27.75,463.0,15.4,DAL
4,5.0,Kawhi Leonard,51,32.2,0.585,16.2,9.0,33.4,3.3,19.7,11.7,26.76,398.8,13.3,LAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,350.0,Dwayne Bacon,39,17.6,0.419,14.6,10.4,19.6,2.3,14.0,7.9,6.15,-44.6,-1.5,CHA
350,351.0,Jerome Robinson,55,13.6,0.439,19.8,10.4,15.4,1.3,13.7,7.5,6.04,-49.8,-1.7,LAC
351,352.0,Allen Crabbe,37,17.6,0.470,14.2,7.3,12.6,1.8,11.0,6.3,6.00,0.0,0.0,ATL
352,353.0,Jacob Evans,29,14.4,0.436,15.3,12.3,17.5,2.3,8.6,5.3,5.55,-30.9,-1.0,GS


In [175]:
d['TEAM'] = d['PLAYER'].apply(team)

In [176]:
d

,RK,PLAYER,GP,MPG,TS%,AST,TO,USG,ORR,DRR,REBR,PER,VA,EWA,TEAM
0,1.0,"Giannis Antetokounmpo, MIL",57,30.9,0.608,17.1,10.8,37.1,7.8,34.5,22.0,31.71,531.8,17.7,MIL
1,2.0,"James Harden, HOU",61,36.7,0.616,18.6,11.3,36.7,3.0,15.3,9.1,28.39,598.5,20.0,HOU
2,3.0,"Anthony Davis, LAL",55,34.3,0.614,11.5,9.0,29.2,7.3,22.1,14.9,28.25,472.1,15.7,LAL
3,4.0,"Luka Doncic, DAL",54,33.3,0.584,23.2,11.2,38.5,4.1,24.7,14.6,27.75,463.0,15.4,DAL
4,5.0,"Kawhi Leonard, LAC",51,32.2,0.585,16.2,9.0,33.4,3.3,19.7,11.7,26.76,398.8,13.3,LAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,350.0,"Dwayne Bacon, CHA",39,17.6,0.419,14.6,10.4,19.6,2.3,14.0,7.9,6.15,-44.6,-1.5,CHA
350,351.0,"Jerome Robinson, LAC/WSH",55,13.6,0.439,19.8,10.4,15.4,1.3,13.7,7.5,6.04,-49.8,-1.7,LAC
351,352.0,"Allen Crabbe, ATL/MIN",37,17.6,0.470,14.2,7.3,12.6,1.8,11.0,6.3,6.00,0.0,0.0,ATL
352,353.0,"Jacob Evans, GS/MIN",29,14.4,0.436,15.3,12.3,17.5,2.3,8.6,5.3,5.55,-30.9,-1.0,GS


In [177]:
d['PLAYER'].apply(player)

0      []
1      []
2      []
3      []
4      []
       ..
349    []
350    []
351    []
352    []
353    []
Name: PLAYER, Length: 354, dtype: object